In [82]:
from random import shuffle

import pandas as pd
import numpy as np

from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('train.csv')
df.head()

,image,species,individual_id
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9
1,000562241d384d.jpg,humpback_whale,1a71fbb72250
2,0007c33415ce37.jpg,false_killer_whale,60008f293a2b
3,0007d9bca26a99.jpg,bottlenose_dolphin,4b00fe572063
4,00087baf5cef7a.jpg,humpback_whale,8e5253662392


In [3]:
df.describe()

,image,species,individual_id
count,51033,51033,51033
unique,51033,30,15587
top,00021adfb725ed.jpg,bottlenose_dolphin,37c7aba965a5
freq,1,9664,400


In [6]:
df.groupby('individual_id')[['image']].count().sort_values('image', ascending=False)

,image
individual_id,
37c7aba965a5,400
114207cab555,168
a6e325d8e924,155
19fbb960f07d,154
c995c043c353,153
...,...
6a0003ef9501,1
6a05336be085,1
6a0ce195bb3f,1


Формируем датасет для chdt

In [127]:
df_chdt = pd.DataFrame()

for _id in tqdm(df['individual_id'].unique()):
    df_id = df[df['individual_id'] == _id] # по каждому id формируем df
    id_specie = df_id['species'].values[0]
    df_id_len = len(df_id)

    df_rest = df[
        (~df['individual_id'].isin([_id]))
        &
        (df['species'] == id_specie)
    ] # df исключая текущий айди но такого же вида
    random_ims = df_rest['image'].sample(n=df_id_len).values # рандомные картинки такого же вида

    init_ims = df_id['image'].values
    random_init_ims = init_ims.copy()
    shuffle(random_init_ims) # мешаем картинки теущего айди
    df_id['image_b'] = random_init_ims
    df_id['label'] = 1 # добавляем их с лейблом 1

    df_id_copy = df_id.copy()
    df_id_copy['image_b'] = random_ims
    df_id_copy['label'] = 0 # добавляем рандомные картинки такого же вида с лейблом 0

    df_concat = pd.concat([df_id, df_id_copy]) # конкатинируем для одного айди

    df_chdt = df_chdt.append(df_concat) # добавляем в общую кучу

100%|██████████| 15587/15587 [03:16<00:00, 79.32it/s]


In [128]:
df_chdt = df_chdt.rename(columns={'image': 'image_a'})
df_chdt.head()

,image_a,species,individual_id,image_b,label
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9,00021adfb725ed.jpg,1
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9,b59f062d32de83.jpg,0
1,000562241d384d.jpg,humpback_whale,1a71fbb72250,ffae18d2939ffc.jpg,1
50972,ffae18d2939ffc.jpg,humpback_whale,1a71fbb72250,000562241d384d.jpg,1
1,000562241d384d.jpg,humpback_whale,1a71fbb72250,7c79741f6d59a2.jpg,0


Формируем датасеты исключая перемешивания айдишников

In [130]:
ids = df_chdt['individual_id'].unique().tolist()
shuffle(ids)

train_df = pd.DataFrame()
validate_df = pd.DataFrame()
test_df = pd.DataFrame()

for _id in tqdm(ids):
    df_id = df_chdt[df_chdt['individual_id'] == _id]
    if train_df.shape[0] <= df_chdt.shape[0] * 0.8: # не больше 80 % на трейн
        train_df = train_df.append(df_id)
    else:
        validate_df = validate_df.append(df_id)
    if validate_df.shape[0] > df_chdt.shape[0] * 0.1: # и примерно по 10-15 % на тест/валидацию
        test_df = test_df.append(df_id)

train_df.shape, validate_df.shape, test_df.shape

100%|██████████| 15587/15587 [01:41<00:00, 153.36it/s]


((81656, 5), (20410, 5), (10214, 5))

In [131]:
train_df.to_csv('../csv/train_df.csv', index=False)
validate_df.to_csv('../csv/validate_df.csv', index=False)
test_df.to_csv('../csv/test_df.csv', index=False)